<div class="alert alert-block alert-info" style="color:black"> <h2>Activity 5: Investigate the time and space (memory) requirements of your two methods</h2>
    You should now have working versions of both breadth-first and (restricted) depth-first search. They already store the number of attempts tested (a measure of runtime), and the code cells that run them print that value out.<br>
    The next step is to compare memory - which is proportional to the maximum size of the open list. 
    <br><br><b>How to get started:</b> Edit your code for both classes:
    <ol>
    <li> Copy-paste <code>update_working_memory()</code> into your <code>BreadthFirstSearch</code> class</li>
    <li> In both your classes add a new parameter <code>self.max_memory</code> with a default value 0 by over-riding the <code>__init__()</code> method of the super class.</ul>
    <li> Override <code>update_working_memory()</code> in both your classes, adding code to:
        <ul>
            <li>check the length of the open_list against <code>self.max_memory</code></li>
            <li> update the value of <code>self.max_memory</code> if the open list has increased in size.</li>
        </ul>
    <li> Copy-paste the testing code from the cells above, then adapt it to test the time and memory needs of your algorithms.</li> 
    </ol>
    <b>Note:</b> this is a <em>Stretch</em> activity so don't worry if you can't complete it easily.
</div>

In [30]:
import sys, os

# Import from the common directory
sys.path.append('../common')
from problem import Problem
from candidatesolution import CandidateSolution
from singlemembersearch import SingleMemberSearch
from combinationproblem import CombinationProblem

class BreadthFirstSearch(SingleMemberSearch):
    """your implementation of depth first search to extend
    the superclass SingleMemberSearch search.
    Adds  a __str__method
    Over-rides the method select_and_move_from_openlist
    to implement the algorithm
    """
    def __init__(
        self,
        problem: Problem,
        constructive: bool = False,
        max_attempts: int = 50,
        minimise=True,
        target_quality=1,

        max_memory=0,
    ):
        """Initialize with an additional max_depth parameter"""
        super().__init__(problem, constructive, max_attempts, minimise, target_quality)
        self.max_memory = max_memory



    def __str__(self):
        return "breadth-first"

    def select_and_move_from_openlist(self) -> CandidateSolution:
        """Implements the breadth-first search algorithm

        Returns
        -------
        next working candidate (solution) taken from open list
        """
        # create a candidate solution variable to hold the next solution
        next_soln = CandidateSolution()

        # ====> insert your pseudo-code and code below here
        if len(self.open_list) == 0:
            return next_soln
        # SelectAndMoveFromOpenList()
        # my_index ← GetFirstIndex(open_list)
        my_index = 0

        # the_candidate ← open_list(my_index)
        next_soln = self.open_list[my_index]

        # RemoveFromOpenList(my_index)
        self.open_list.pop(my_index)

        # <==== insert your pseudo-code and code above here
        return next_soln

    def update_working_memory(self, neighbour: CandidateSolution,reason:str):
        """Update what we have learned about the problem
        after evaluating a new candidate
        Could have left this code in the main loop
        but separating it out makes it easier to read.
        """
        # === Pseudocode: IF status IS AtGoal THEN Return(SUCCESS)
        # for decision problems this means quality==1
        if neighbour.quality == self.target_quality:
            self.result = neighbour.variable_values
            self.solved = True

        # === Pseudocode: ELSE IF status IS BREAKS_CONSTRAINTS THEN
        elif reason != "":
            self.runlog += (
                f"discarding invalid solution {neighbour.variable_values} "
                f"because    {reason}\n"
            )
            # PS AppendToClosedList(neighbour)
            self.closed_list.append(neighbour)

        # === Pseudocode: ELSE AppendToOpenList(neighbour)
        else:
            self.runlog += (
                "adding solution to openlist"
                f": to examine later: {neighbour.variable_values}\t"
                f" quality {neighbour.quality}\n"
            )
            self.open_list.append(neighbour)
        # Update max_memory
        if len(self.open_list) > self.max_memory:
            self.max_memory = len(self.open_list)

In [31]:
class RestrictedDepthFirstSearch(SingleMemberSearch):
    """Implementation of depth first search with a maximum depth limit
    Extends the SingleMemberSearch class.
    Adds a __str__ method
    Over-rides the methods:
    - __init__ to add max_depth parameter
    - select_and_move_from_openlist to implement depth-first selection
    - update_working_memory to restrict depth of search
    """

    def __init__(
        self,
        problem: Problem,
        constructive: bool = False,
        max_attempts: int = 50,
        minimise=True,
        target_quality=1,
        max_depth=10,
        max_memory=0,
    ):
        """Initialize with an additional max_depth parameter"""
        super().__init__(problem, constructive, max_attempts, minimise, target_quality)
        self.max_depth = max_depth
        self.max_memory = max_memory

    def __str__(self):
        return "restricted-depth-first"

    def select_and_move_from_openlist(self) -> CandidateSolution:
        """Implements the depth-first search algorithm

        Returns
        -------
        next working candidate (solution) taken from open list
        """
        # create a candidate solution variable to hold the next solution
        next_soln = CandidateSolution()
        # check if the open list is not empty
        if len(self.open_list) == 0:
            return next_soln

        # SelectAndMoveFromOpenList()
        # my_index ← GetLastIndex(open_list)
        my_index = len(self.open_list) - 1

        # the_candidate ← open_list(my_index)
        next_soln = self.open_list[my_index]

        # RemoveFromOpenList(my_index)
        self.open_list.pop(my_index)

        # Return(the_candidate)
        return next_soln

    def update_working_memory(self, neighbour: CandidateSolution,reason:str):
        """Update what we have learned about the problem
        after evaluating a new candidate
        Could have left this code in the main loop
        but separating it out makes it easier to read.
        """
        # === Pseudocode: IF status IS AtGoal THEN Return(SUCCESS)
        # for decision problems this means quality==1
        if neighbour.quality == self.target_quality:
            self.result = neighbour.variable_values
            self.solved = True

        # === Pseudocode: ELSE IF status IS BREAKS_CONSTRAINTS THEN
        elif reason != "":
            self.runlog += (
                f"discarding invalid solution {neighbour.variable_values} "
                f"because    {reason}\n"
            )
            # PS AppendToClosedList(neighbour)
            self.closed_list.append(neighbour)

        # === Pseudocode: ELSE AppendToOpenList(neighbour)
        elif len(neighbour.variable_values) < self.max_depth:
            self.runlog += (
                "adding solution to openlist"
                f": to examine later: {neighbour.variable_values}\t"
                f" quality {neighbour.quality}\n"
            )
            self.open_list.append(neighbour)
        # Update max_memory
        if len(self.open_list) > self.max_memory:
            self.max_memory = len(self.open_list)


In [32]:
#next line not necessary if you've run the cell above - but python  will ignore it
from foxchickengrain import FoxChickenGrain

# Create a FoxChickenGrain problem
myproblem = FoxChickenGrain()

# Create a depth-first and breadth-first search algorithm
my_depth_search = RestrictedDepthFirstSearch(myproblem, constructive=True, max_attempts=1000, max_memory=00)
my_breadth_search = BreadthFirstSearch(myproblem, constructive=True, max_attempts=500, max_memory=00)

print_runlog = True  # you might want to turn this on for debugging

for algorithm in (my_depth_search, my_breadth_search):
    print(f"\n Using the algorithm {algorithm.__str__()}")
    found = algorithm.run_search()
    if found:
        print(
            f"\t Solved after {algorithm.trials} attempts:\n"
            f"\t Solution is{myproblem.display(algorithm.result)}\n"
            f"\t Memory used: {algorithm.max_memory}"
        )
    else:
        print("\t Problem not solved in time allowed")

        if print_runlog:
            print(algorithm.runlog)


 Using the algorithm restricted-depth-first
	 Solved after 644 attempts:
	 Solution is->bc_01->bc_10->bc_01->b_10->bf_01->bc_10->bg_01->b_10->bc_01
	 Memory used: 10

 Using the algorithm breadth-first
	 Solved after 404 attempts:
	 Solution is->bc_01->b_10->bg_01->bc_10->bf_01->b_10->bc_01
	 Memory used: 59


In [22]:
import workbook2_utils as wb2
display(wb2.q12)
display(wb2.q13)